### Import and install the lib

In [111]:
#!conda config --remove channels conda-forge
#!conda config --add channels conda-forge


CondaKeyError: 'channels': 'conda-forge' is not in the 'channels' key of the config file



In [116]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import re

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [117]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
try:
    print("Installing geopy...\n")
    #!conda install -c conda-forge geopy --yes
    !pip install geopy
    print("geopy has been successfully installed!\n")
except:
    print("ERROR: could not install geopy!\n")

import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Installing geopy...

geopy has been successfully installed!



In [119]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
try:
    print("Installing folium...\n")
    #!conda install -c conda-forge folium --yes
    !pip install folium
    print("folium has been successfully installed!\n")
except:
    print("ERROR: could not install folium!\n")

import folium # map rendering library

print('Libraries imported.')

Installing folium...

folium has been successfully installed!

Libraries imported.


### scrape the wiki page

In [120]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = urlopen(url)

soup = BeautifulSoup(html,'html.parser')

soup_1 = soup.find_all('table')

In [121]:
tables = soup.find('table')
#tables.find_all('tr')

### Extract the table

In [122]:
#Create array to hold the data we extract
data_postal_code = []
data_borough = []
data_neighbour = []

rows = tables.find_all('tr')
print('there are %d rows'%(len(rows)))
for row in rows:
    cells = row.find_all('td')

    #print('there are %d cells'%(len(cells)))
    if len(cells) > 1:
        postal_code = cells[0]
        data_postal_code.append(postal_code.text.strip())

        borough = cells[1]
        data_borough.append(borough.text.strip())

        neighbour = cells[2]
        #print(neighbour.text)
        data_neighbour.append(neighbour.text.strip())
        
    #data_neighbour

there are 181 rows


### Prepare the dataframe

In [123]:
df = pd.DataFrame(data_postal_code, columns = ['PostalCode'])
df['Borough'] = data_borough
df['Neighbourhood'] = data_neighbour
num = df.shape
print('origial df has %d rows'%(num[0]))
#df.drop(df['Borough'] == 'Not assigned', inplace = True)
df = df[~df['Borough'].str.contains("Not assigned")]
df.head(20)

origial df has 180 rows


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [124]:
df.shape

(103, 3)

### Group the Neighbourhood

In [125]:
df.groupby(['PostalCode','Borough'])['Neighbourhood'].agg(','.join).reset_index()
df.head(20)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


### Show the shape of the dataframe

In [126]:
df.shape

(103, 3)

### Get the latitude and longitude of Toronto

In [129]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [130]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

body = client_ac391cda40084063a272c3b262c6a5d2.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-gamtkiidehvnk2',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_lat_long = pd.read_csv(body)
df_lat_long.rename(columns = {'Postal Code':'PostalCode'},inplace = True)
df_lat_long.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [131]:
df_lat_long.shape


(103, 3)

In [132]:
df_geo = pd.merge(df, df_lat_long, on = 'PostalCode')
df_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [133]:
df_geo.shape

(103, 5)

### Cluster Neighborhoods

In [149]:
# set number of clusters
kclusters = 5

X = df_geo['Latitude']
Y = df_geo['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow','black']
df_geo['Cluster'] = clusters

#df_geo['Cluster']

### Create a map of Toronto with neighborhoods superimposed on top.

In [148]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, cluster in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Cluster']):
    label = '{}, {}'.format(cluster, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=colors[cluster-1],
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto